In [1]:
import pandas as pd

# Define instruction characteristics
instructions = [
    {"pc": "0x100", "op": "lw", "dest": "x1", "src1": "x1", "src2": None, "imm": 4, "latency": 4},
    {"pc": "0x104", "op": "xor", "dest": "x3", "src1": "x0", "src2": "x1", "imm": None, "latency": 1},
    {"pc": "0x108", "op": "sra", "dest": "x1", "src1": "x3", "src2": "x2", "imm": None, "latency": 2},
    {"pc": "0x10c", "op": "sub", "dest": "x1", "src1": "x1", "src2": "x0", "imm": None, "latency": 1},
    {"pc": "0x110", "op": "addi", "dest": "x2", "src1": "x0", "src2": None, "imm": 2, "latency": 1},
    {"pc": "0x114", "op": "xor", "dest": "x2", "src1": "x3", "src2": "x0", "imm": None, "latency": 1},
]

# Pipeline stages
pipeline_stages = ["R", "Di", "I", "C", "Rt"]

# Track state of each instruction through each cycle
class Instruction:
    def __init__(self, info):
        self.pc = info["pc"]
        self.op = info["op"]
        self.dest = info["dest"]
        self.src1 = info["src1"]
        self.src2 = info["src2"]
        self.imm = info["imm"]
        self.latency = info["latency"]
        self.current_stage = "R"  # Start at Rename
        self.stage_time = 0       # Track time in the current stage
        self.completed = False    # Track if the instruction is retired

    def advance_stage(self):
        """Advance instruction to the next stage if time in the current stage is complete"""
        # Define stage durations (Issue stage has unique latency)
        stage_durations = {"R": 1, "Di": 1, "I": self.latency, "C": 1, "Rt": 1}
        
        if self.stage_time >= stage_durations[self.current_stage]:
            # Move to next stage if stage time is complete
            next_stage = pipeline_stages[(pipeline_stages.index(self.current_stage) + 1)]
            self.current_stage = next_stage
            self.stage_time = 0  # Reset time for the new stage
        else:
            self.stage_time += 1

    def get_stage_display(self):
        """Display current stage and time in stage"""
        return self.current_stage * (self.stage_time + 1)

# Initialize instructions
instruction_objects = [Instruction(instr) for instr in instructions]

# Cycle tracking
cycles = []
cycle_count = 2  # Start from cycle 2 as per instructions

# Run simulation until all instructions have retired
while any(not instr.completed for instr in instruction_objects):
    cycle_data = {"Cycle": cycle_count}
    for instr in instruction_objects:
        # Advance each instruction's stage if it's not yet retired
        if instr.current_stage != "Rt" or instr.stage_time < 1:
            instr.advance_stage()
        
        # Display current stage for each instruction in cycle data
        cycle_data[instr.pc] = instr.get_stage_display()
        
        # Mark as completed in ROB if reached Rt stage
        if instr.current_stage == "Rt" and instr.stage_time == 1:
            instr.completed = True

    cycles.append(cycle_data)
    cycle_count += 1

# Convert cycle data to DataFrame for display
cycle_df = pd.DataFrame(cycles).fillna("")

# Reorder Buffer (ROB) state at end of simulation
def create_rob(instruction_objects):
    rob_data = [{
        "PC": instr.pc,
        "DestReg": instr.dest,
        "Completed": "Yes" if instr.completed else "No"
    } for instr in instruction_objects]
    return pd.DataFrame(rob_data)

# Display outputs
print("Cycle-by-Cycle Instruction Table")
print(cycle_df)

rob_df = create_rob(instruction_objects)
print("\nReorder Buffer (ROB) Table at End of Execution")
print(rob_df)


Cycle-by-Cycle Instruction Table
    Cycle  0x100 0x104 0x108 0x10c 0x110 0x114
0       2     RR    RR    RR    RR    RR    RR
1       3     Di    Di    Di    Di    Di    Di
2       4   DiDi  DiDi  DiDi  DiDi  DiDi  DiDi
3       5      I     I     I     I     I     I
4       6     II    II    II    II    II    II
5       7    III     C   III     C     C     C
6       8   IIII    CC     C    CC    CC    CC
7       9  IIIII    Rt    CC    Rt    Rt    Rt
8      10      C  RtRt    Rt  RtRt  RtRt  RtRt
9      11     CC  RtRt  RtRt  RtRt  RtRt  RtRt
10     12     Rt  RtRt  RtRt  RtRt  RtRt  RtRt
11     13   RtRt  RtRt  RtRt  RtRt  RtRt  RtRt

Reorder Buffer (ROB) Table at End of Execution
      PC DestReg Completed
0  0x100      x1       Yes
1  0x104      x3       Yes
2  0x108      x1       Yes
3  0x10c      x1       Yes
4  0x110      x2       Yes
5  0x114      x2       Yes
